In [ ]:
import collections.abc
#hyper needs the four following aliases to be done manually.
collections.Iterable = collections.abc.Iterable
collections.Mapping = collections.abc.Mapping
collections.MutableSet = collections.abc.MutableSet
collections.MutableMapping = collections.abc.MutableMapping

In [ ]:
import numpy as np
import pandas as pd
#pd.set_option('max_columns', None)
#pd.set_option('max_rows', 90)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 80)

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

from sklearn.neighbors import KNeighborsRegressor
import scipy.stats
from sklearn.preprocessing import StandardScaler
from pycaret.regression import setup, compare_models
from sklearn.model_selection import KFold, cross_val_score

from catboost import CatBoostRegressor
from sklearn.linear_model import BayesianRidge, HuberRegressor, Ridge, OrthogonalMatchingPursuit
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor

import optuna

In [ ]:
train0 = pd.read_csv('train.csv')
test0 = pd.read_csv('test.csv')
sample_submission = pd.read_csv("sample_submission.csv")

In [ ]:
train0

In [ ]:
train0.info()

In [ ]:
test0

In [ ]:
target = train0['SalePrice']
testID = train0['Id']

train1 = train0.drop(['Id', 'SalePrice'], axis=1)
test1 = test0.drop(['Id'], axis=1)
data1 = pd.concat([train1, test1], axis=0).reset_index(drop=True)

In [ ]:
data1

End of stage 1

In [ ]:
data2 = data1.copy()

In [ ]:
data2['MSSubClass'] = data2['MSSubClass'].astype(str)

In [ ]:
data2['MSSubClass']

Fill categorical columns

In [ ]:
cat_col = []
num_col = []
for col in data2.columns:
    if data2[col].dtype in ['int32', 'int64', 'float32', 'float64']:
        num_col.append(col)
    else:
        cat_col.append(col)    

In [ ]:
nans = []
for col in data2[cat_col]:
    if data2[col].isna().sum() >= 1:
        nans.append(col)

In [ ]:
not_nan= ['BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','FireplaceQu',
 'GarageType','GarageFinish', 'GarageQual', 'GarageCond','PoolQC','Fence','MiscFeature','Alley']

In [ ]:
non = []
for a in nans :
  if a not in not_nan:
    non.append(a)

In [ ]:
non

In [ ]:
for col in not_nan:
    data2[col].fillna("none", inplace=True)

In [ ]:
data2[not_nan].isna().sum()

In [ ]:
for col in non:
    data2[col] = data2[col].fillna(data2[col].mode()[0])

In [ ]:
data2[non].isna().sum()

In [ ]:
data2[cat_col].isna().sum().sum()

Stage 2 Completed

stage 3 deal with nons in numeric

In [ ]:
data3 = data2.copy()

In [ ]:
numeics = data3[num_col].iloc[:,(data3[num_col].isna().sum()==0).values]

In [ ]:
num_not_nan = numeics.columns

In [ ]:
num_not_nan

In [ ]:
num_nan = []
for a in num_col :
  if a not in num_not_nan:
    num_nan.append(a)

In [ ]:
num_nan

In [ ]:
from sklearn.impute import KNNImputer

In [ ]:
data3.isna().sum()

In [ ]:
impute_knn = KNNImputer(n_neighbors=10)
data3_1 = impute_knn.fit_transform(data3[num_col])

In [ ]:
data3_2 = pd.DataFrame(data3_1, columns= num_col)

In [ ]:
data3_2

In [ ]:
data3

In [ ]:
data3_3 = data3[cat_col]

Stage 3 Compeleted

Transformation

In [ ]:
data4 = pd.concat([data3_2,data3_3],axis=1 ,ignore_index=False)

In [ ]:
data4.isna().sum().sum()

In [ ]:
data5 = data4.copy()

In [ ]:
data5[num_col].hist(bins=10,figsize=(25,25))
plt.show()

In [ ]:
data5[num_col].describe()

In [ ]:
data6 = data5.copy()

In [ ]:
from scipy.stats import skew

In [ ]:
data6[num_col].describe()

In [ ]:
for col in num_col:
    if np.abs(skew(data6[col])) >0.5:
       data6[col]= np.sqrt(data6[col])

In [ ]:
for col in num_col:
   print(skew(data5[col]))

In [ ]:
for col in num_col:
   print(skew(data6[col]))

In [ ]:
data6[num_col].hist(bins=10,figsize=(20,20))
plt.show()

Encoding categoricals And Standardition

In [ ]:
from sklearn.pipeline import Pipeline #,FeatureUnion
"""from sklearn.preprocessing import StandardScaler, OneHotEncoder
"""
from sklearn_features.transformers import DataFrameSelector

In [ ]:
data7_1 = pd.get_dummies(data6[cat_col])

In [ ]:
data7_1

In [ ]:
stepsnum = [("selector", DataFrameSelector(num_col)),
         ("scaler", StandardScaler())]

pipenum = Pipeline(stepsnum)

#stepscat = [("selector", DataFrameSelector(cat_col)),
 #                ("ohe", OneHotEncoder())]
                 
#pipecat = Pipeline(stepscat)

#transformer_list=[("num", pipenum),("cat", pipecat)]
#pipel = FeatureUnion(transformer_list)

In [ ]:
data7_2 = pipenum.fit_transform(data6)

In [ ]:
data7_3 =pd.DataFrame(data7_2, columns=num_col)

In [ ]:
data7_4 = pd.concat([data7_3, data7_1] , axis=1)

In [ ]:
data7_4

Stage 7 is Compeleted

Target Transformation

In [ ]:
target